In [2]:
import random
import json

# Creating sample training data based on the proposed flight booking schema

# Sample schema context for the system's message
schema_context = """
You are an advanced text-to-SQL model tasked with generating SQL queries for a PostgreSQL database related to flight bookings. Your responses must adhere to PostgreSQL syntax and consider the specificities of its SQL implementation.
Your job involves interpreting user queries about flight schedules, bookings, passenger details, and airport information, and responding with appropriate SQL queries based on the provided PostgreSQL schema.

### Context:
CREATE TABLE Flights (
    flight_id INTEGER PRIMARY KEY,
    airline VARCHAR,
    origin VARCHAR,
    destination VARCHAR,
    departure_time TIMESTAMP,
    arrival_time TIMESTAMP,
    price DECIMAL
);

CREATE TABLE Bookings (
    booking_id INTEGER PRIMARY KEY,
    flight_id INTEGER,
    passenger_id INTEGER,
    booking_date TIMESTAMP,
    status VARCHAR
);

CREATE TABLE Passengers (
    passenger_id INTEGER PRIMARY KEY,
    name VARCHAR,
    email VARCHAR,
    phone VARCHAR
);

CREATE TABLE Airports (
    airport_code VARCHAR PRIMARY KEY,
    name VARCHAR,
    city VARCHAR,
    country VARCHAR
);
"""

# # List of sample user queries and assistant SQL queries
#  = [
#     {
#         "role": "system",
#         "content": schema_context
#     },
#     {
#         "role": "user",
#         "content": "Find all flights from New York to London on July 15, 2023."
#     },
#     {
#         "role": "assistant",
#         "content": "SELECT * FROM Flights WHERE origin = 'New York' AND destination = 'London' AND DATE(departure_time) = '2023-07-15'"
#     },
#     {
#         "role": "user",
#         "content": "How can I cancel a booking with booking ID 102?"
#     },
#     {
#         "role": "assistant",
#         "content": "UPDATE Bookings SET status = 'cancelled' WHERE booking_id = 102"
#     },
#     {
#         "role": "user",
#         "content": "List all passengers who have booked a flight to Tokyo."
#     },
#     {
#         "role": "assistant",
#         "content": "SELECT p.name, p.email FROM Passengers p JOIN Bookings b ON p.passenger_id = b.passenger_id JOIN Flights f ON b.flight_id = f.flight_id WHERE f.destination = 'Tokyo'"
#     }
# ]
# sample_interactions
# # Format data into JSONL for training
# training_data = [{'messages': [sample_interactions[0], sample_interactions[1], sample_interactions[2]]},
#                  {'messages': [sample_interactions[0], sample_interactions[3], sample_interactions[4]]},
#                  {'messages': [sample_interactions[0], sample_interactions[5], sample_interactions[6]]}]



# Extending the schema context with specifics about Can Tho airport (assumed code CTH)
extended_schema_context = schema_context + "\n-- Note: Can Tho airport code is 'CTH'"

# Sample user queries for flights from Can Tho
queries_from_can_tho = [
    "Find flights from Can Tho to Hanoi.",
    "Show all morning flights from Can Tho tomorrow.",
    "List the cheapest flight from Can Tho to Ha Noi City next Monday.",
    "Get all direct flights from Can Tho to Da Nang this weekend.",
    "Find all available seats on flights from Can Tho to Phu Quoc on July 4, 2023.",
    "Find flights from Can Tho to Bangkok.",
    "List direct flights from Can Tho to Singapore next weekend.",
    "Display flights leaving Can Tho after 6 PM today.",
    "Find return flights to Can Tho from Da Nang arriving before noon.",
    "Search for the lowest fare from Can Tho to Kuala Lumpur on July 20, 2023.",
    "Get details of Vietnam Airlines flights from Can Tho to Seoul.",
    "Check availability for flights from Can Tho to Tokyo with less than one stop.",
    "Show flights from Can Tho with a layover in Hanoi.",
    "Find all flights from Can Tho under $300.",
    "List all flights from Can Tho to Nha Trang departing next Friday."
]

# Corresponding SQL queries
sql_queries_from_can_tho = [
    "SELECT * FROM Flights WHERE origin = 'CTH' AND destination = 'HAN'",  # No change needed.
    "SELECT * FROM Flights WHERE origin = 'CTH' AND EXTRACT(HOUR FROM departure_time) < 12 AND departure_time::date = CURRENT_DATE + INTERVAL '1 day'",
    "SELECT * FROM Flights WHERE origin = 'CTH' AND destination = 'HAN' AND departure_time::date = CURRENT_DATE + INTERVAL '1 day' * (7 - EXTRACT(ISODOW FROM CURRENT_DATE) + 1) ORDER BY price ASC LIMIT 1",
    "SELECT * FROM Flights WHERE origin = 'CTH' AND destination = 'DAD' AND departure_time::date BETWEEN CURRENT_DATE + INTERVAL '1 day' * ((6 - EXTRACT(ISODOW FROM CURRENT_DATE) + 7) % 7) AND CURRENT_DATE + INTERVAL '1 day' * ((7 - EXTRACT(ISODOW FROM CURRENT_DATE) + 7) % 7)",
    "SELECT f.*, COUNT(b.booking_id) as available_seats FROM Flights f LEFT JOIN Bookings b ON f.flight_id = b.flight_id WHERE f.origin = 'CTH' AND f.destination = 'Phu Quoc' AND departure_time::date = '2023-07-04' GROUP BY f.flight_id",
    "SELECT * FROM Flights WHERE origin = 'CTH' AND destination = 'Bangkok'",  # No change needed.
    "SELECT * FROM Flights WHERE origin = 'CTH' AND destination = 'Singapore' AND departure_time::date BETWEEN CURRENT_DATE + INTERVAL '1 day' * ((6 - EXTRACT(ISODOW FROM CURRENT_DATE) + 7) % 7) AND CURRENT_DATE + INTERVAL '1 day' * ((7 - EXTRACT(ISODOW FROM CURRENT_DATE) + 7) % 7)",
    "SELECT * FROM Flights WHERE origin = 'CTH' AND EXTRACT(HOUR FROM departure_time) >= 18",
    "SELECT * FROM Flights WHERE destination = 'CTH' AND origin = 'Da Nang' AND EXTRACT(HOUR FROM arrival_time) < 12",
    "SELECT * FROM Flights WHERE origin = 'CTH' AND destination = 'Kuala Lumpur' AND departure_time::date = '2023-07-20' ORDER BY price ASC LIMIT 1",
    "SELECT * FROM Flights WHERE airline = 'Vietnam Airlines' AND origin = 'CTH' AND destination = 'Seoul'",  # No change needed.
    "SELECT * FROM Flights WHERE origin = 'CTH' AND destination = 'Tokyo'",  # No change needed.
    "SELECT * FROM Flights WHERE origin = 'CTH' AND destination = 'HAN'",  # Needs review; logic might be incorrect.
    "SELECT * FROM Flights WHERE origin = 'CTH' AND price < 300",  # No change needed.
    "SELECT * FROM Flights WHERE origin = 'CTH' AND departure_time::date = CURRENT_DATE + INTERVAL '1 day' * ((5 - EXTRACT(ISODOW FROM CURRENT_DATE) + 7) % 7) + INTERVAL '7 days'"  # Corrected for next Friday.
]

# Generating 100 records with random selections of the above queries
extended_training_data = []
for _ in range(100):
    idx = random.randint(0, len(queries_from_can_tho) - 1)
    interaction = {
        "role": "system",
        "content": extended_schema_context
    }
    user_query = {
        "role": "user",
        "content": queries_from_can_tho[idx]
    }
    assistant_response = {
        "role": "assistant",
        "content": sql_queries_from_can_tho[idx]
    }
    extended_training_data.append({'messages': [interaction, user_query, assistant_response]})

# Save to a new JSONL file
extended_file_path = 'fine_tuning_sql_tune_tourism_assistant.jsonl'
with open(extended_file_path, 'w') as extended_file:
    for entry in extended_training_data:
        json.dump(entry, extended_file)
        extended_file.write('\n')

extended_file_path


'fine_tuning_sql_tune_tourism_assistant.jsonl'